### 1. Create Train Script 

In [84]:
%%file train
#!/usr/bin/env python

from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import pickle
import os


np.random.seed(123)

# Define paths for Model Training inside Container.
INPUT_PATH = '/opt/ml/input/data'
OUTPUT_PATH = '/opt/ml/output'
MODEL_PATH = '/opt/ml/model'
PARAM_PATH = '/opt/ml/input/config/hyperparameters.json'

# Training data sitting in S3 will be copied to this location during training when used with File MODE.
TRAIN_DATA_PATH = f'{INPUT_PATH}/train'
TEST_DATA_PATH = f'{INPUT_PATH}/test'

def train():
    print("------- [STARTING TRAINING] -------")
    train_df = pd.read_csv(os.path.join(TRAIN_DATA_PATH, 'train.csv'), names=['class', 'mass', 'width', 'height', 'color_score'])
    train_df.head()
    X_train = train_df[['mass', 'width', 'height', 'color_score']]
    y_train = train_df['class']
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)
    # Save the trained Model inside the Container
    with open(os.path.join(MODEL_PATH, 'model.pkl'), 'wb') as out:
        pickle.dump(knn, out)
    print("------- [TRAINING COMPLETE!] -------")
    
    print("------- [STARTING EVALUATION] -------")
    test_df = pd.read_csv(os.path.join(TEST_DATA_PATH, 'test.csv'), names=['class', 'mass', 'width', 'height', 'color_score'])
    X_test = train_df[['mass', 'width', 'height', 'color_score']]
    y_test = train_df['class']
    acc = knn.score(X_test, y_test)
    print('Accuracy = {:.2f}%'.format(acc * 100))
    print("------- [EVALUATION DONE!] -------")

if __name__ == '__main__':
    train()

Overwriting train


### 2. Create Serve Script

In [85]:
%%file serve
#!/usr/bin/env python

from flask import Flask, Response, request
from io import StringIO
import pandas as pd
import logging
import pickle
import os


app = Flask(__name__)

MODEL_PATH = '/opt/ml/model'

# Singleton Class for holding the Model
class Predictor:
    model = None
    
    @classmethod
    def load_model(cls):
        print('[LOADING MODEL]')
        if cls.model is None:
            with open(os.path.join(MODEL_PATH, 'model.pkl'), 'rb') as file_:
                cls.model = pickle.load(file_)
        print('MODEL LOADED!')
        return cls.model
    
    @classmethod
    def predict(cls, X):
        clf = cls.load_model()
        return clf.predict(X)

@app.route('/ping', methods=['GET'])
def ping():
    print('[HEALTH CHECK]')
    model = Predictor.load_model()
    status = 200
    if model is None:
        status = 404
    return Response(response={"HEALTH CHECK": "OK"}, status=status, mimetype='application/json')

@app.route('/invocations', methods=['POST'])
def invoke():
    data = None

    # Transform Payload in CSV to Pandas DataFrame.
    if request.content_type == 'text/csv':
        data = request.data.decode('utf-8')
        data = StringIO(data)
        data = pd.read_csv(data, header=None)
    else:
        return flask.Response(response='This Predictor only supports CSV data', status=415, mimetype='text/plain')

    logging.info('Invoked with {} records'.format(data.shape[0]))
    
    predictions = Predictor.predict(data)

    # Convert from numpy back to CSV
    out = StringIO()
    pd.DataFrame({'results': predictions}).to_csv(out, header=False, index=False)
    result = out.getvalue()

    return Response(response=result, status=200, mimetype='text/csv')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)

Overwriting serve


### 3. Build a Docker Image and Push to ECR

In [86]:
%%sh

# Assign a name for your Docker image.
image_name=byoc-sklearn
echo "Image Name: ${image_name}" 

# Retrieve AWS Account.
account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-east-1 if none defined).
region=$(aws configure get region)
region=${region:-us-east-1}

echo "Account: ${account}" 
echo "Region: ${region}"

repository="${account}.dkr.ecr.${region}.amazonaws.com"
echo "Repository: ${repository}" 

image="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:latest"
echo "Image URI: ${image}" 

# If the repository does not exist in ECR, create it.
aws ecr describe-repositories --repository-names ${image_name} > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name ${image_name} > /dev/null
fi

# Get the login command from ECR and execute it directly.
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${repository}

# Build the docker image locally with the image name and tag it.
docker build -t ${image_name} .
docker tag ${image_name} ${image}

# Finally, push image to ECR with the full image name.
docker push ${image}

Image Name: byoc-sklearn
Account: 892313895307
Region: us-east-1
Repository: 892313895307.dkr.ecr.us-east-1.amazonaws.com
Image URI: 892313895307.dkr.ecr.us-east-1.amazonaws.com/byoc-sklearn:latest
Login Succeeded

Step 1/8 : FROM python:3.7
 ---> 5b86e11778a2
Step 2/8 : COPY requirements.txt ./
 ---> Using cache
 ---> 8623cb69764a
Step 3/8 : RUN pip install --no-cache-dir -r requirements.txt
 ---> Using cache
 ---> 00be6a106a8c
Step 4/8 : COPY train /usr/local/bin
 ---> Using cache
 ---> f55d18c34b89
Step 5/8 : RUN chmod +x /usr/local/bin/train
 ---> Using cache
 ---> aae62ce0c43b
Step 6/8 : COPY serve /usr/local/bin
 ---> Using cache
 ---> d9408249ae77
Step 7/8 : RUN chmod +x /usr/local/bin/serve
 ---> Using cache
 ---> 04fc001c0b7c
Step 8/8 : EXPOSE 8080
 ---> Using cache
 ---> 6990c97b2383
Successfully built 6990c97b2383
Successfully tagged byoc-sklearn:latest
The push refers to repository [892313895307.dkr.ecr.us-east-1.amazonaws.com/byoc-sklearn]
032f1a03bf08: Preparing
053f06468

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



### Imports 

In [87]:
from sagemaker.predictor import csv_serializer
import pandas as pd
import sagemaker

### Essentials

In [88]:
role = sagemaker.get_execution_role()
session = sagemaker.Session()
account = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name
image_name = 'byoc-sklearn'
image_uri = f'{account}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest'

### Train (Local Mode)

In [89]:
model = sagemaker.estimator.Estimator(
    image_name=image_uri,
    role=role,
    train_instance_count=1,
    train_instance_type='local',
    sagemaker_session=None
)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [90]:
model.fit({'train': 'file://.././DATA/train/train.csv', 'test': 'file://.././DATA/test/test.csv'})

Creating tmp815252o0_algo-1-n7amc_1 ... 
Attaching to tmp815252o0_algo-1-n7amc_12mdone
algo-1-n7amc_1  | ------- [STARTING TRAINING] -------
algo-1-n7amc_1  | ------- [TRAINING COMPLETE!] -------
algo-1-n7amc_1  | ------- [STARTING EVALUATION] -------
algo-1-n7amc_1  | Accuracy = 97.73%
algo-1-n7amc_1  | ------- [EVALUATION DONE!] -------
tmp815252o0_algo-1-n7amc_1 exited with code 0
Aborting on container exit...
===== Job Complete =====


### Deploy (Locally)

In [6]:
predictor = model.deploy(1, 'local', endpoint_name='byoc-sklearn', serializer=csv_serializer)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


Attaching to tmpm093izjl_algo-1-g5u49_1
algo-1-g5u49_1  |  * Serving Flask app "serve" (lazy loading)
algo-1-g5u49_1  |  * Environment: production
algo-1-g5u49_1  |    WARNING: This is a development server. Do not use it in a production deployment.
algo-1-g5u49_1  |    Use a production WSGI server instead.
algo-1-g5u49_1  |  * Debug mode: off
algo-1-g5u49_1  |  * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
algo-1-g5u49_1  | [HEALTH CHECK]
algo-1-g5u49_1  | [LOADING MODEL]
algo-1-g5u49_1  | MODEL LOADED!
algo-1-g5u49_1  | 172.18.0.1 - - [04/Nov/2020 00:16:46] "GET /ping HTTP/1.1" 200 -
!

### Evaluate Real Time Inference (Locally)

In [24]:
df = pd.read_csv('.././DATA/test/test.csv', header=None)
test_df = df.sample(1)

In [25]:
test_df

,0,1,2,3,4
5,3,1.021429,1.117647,0.8,0.441176


In [26]:
test_df.drop(test_df.columns[[0]], axis=1, inplace=True)
test_df

,1,2,3,4
5,1.021429,1.117647,0.8,0.441176


In [27]:
test_df.values

array([[1.02142857, 1.11764706, 0.8       , 0.44117647]])

In [28]:
prediction = predictor.predict(test_df.values).decode('utf-8').strip()

algo-1-g5u49_1  | [LOADING MODEL]
algo-1-g5u49_1  | MODEL LOADED!
algo-1-g5u49_1  | 172.18.0.1 - - [04/Nov/2020 00:18:22] "POST /invocations HTTP/1.1" 200 -
algo-1-g5u49_1  | INFO:werkzeug:172.18.0.1 - - [04/Nov/2020 00:18:22] "POST /invocations HTTP/1.1" 200 -


In [29]:
prediction

'3'

### Train (using SageMaker)

In [45]:
WORK_DIRECTORY = '.././DATA'

train_data_s3_pointer = session.upload_data(f'{WORK_DIRECTORY}/train', key_prefix='byoc-sklearn/train')
test_data_s3_pointer = session.upload_data(f'{WORK_DIRECTORY}/test', key_prefix='byoc-sklearn/test')

In [46]:
train_data_s3_pointer

's3://sagemaker-us-east-1-892313895307/byoc-sklearn/train'

In [47]:
test_data_s3_pointer

's3://sagemaker-us-east-1-892313895307/byoc-sklearn/test'

In [48]:
model = sagemaker.estimator.Estimator(
    image_name=image_uri,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.m5.xlarge',
    sagemaker_session=session  # ensure the session is set to session
)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [50]:
model.fit({'train': train_data_s3_pointer, 'test': test_data_s3_pointer})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-11-04 17:32:19 Starting - Starting the training job...
2020-11-04 17:32:21 Starting - Launching requested ML instances.........
2020-11-04 17:33:52 Starting - Preparing the instances for training...
2020-11-04 17:34:27 Downloading - Downloading input data......
2020-11-04 17:35:31 Training - Downloading the training image...
2020-11-04 17:36:19 Uploading - Uploading generated training model
2020-11-04 17:36:19 Completed - Training job completed
------- [STARTING TRAINING] -------
------- [TRAINING COMPLETE!] -------
------- [STARTING EVALUATION] -------
Accuracy = 97.73%
------- [EVALUATION DONE!] -------
Training seconds: 112
Billable seconds: 112


### Deploy Trained Model as SageMaker Endpoint

In [52]:
predictor = model.deploy(1, 'ml.m5.xlarge', endpoint_name='byoc-sklearn', serializer=csv_serializer)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-----------!

### Real Time Inference using Deployed Endpoint

In [54]:
df = pd.read_csv('.././DATA/test/test.csv', header=None)
test_df = df.sample(1)

In [55]:
test_df.drop(test_df.columns[[0]], axis=1, inplace=True)
test_df

,1,2,3,4
12,0.342857,0.382353,0.553846,0.970588


In [56]:
test_df.values

array([[0.34285714, 0.38235294, 0.55384615, 0.97058824]])

In [57]:
prediction = predictor.predict(test_df.values).decode('utf-8').strip()

In [58]:
prediction

'0'

### Batch Transform (Batch Inference) using Trained SageMaker Model

In [68]:
bucket_name = session.default_bucket()
output_path = f's3://{bucket_name}/byoc-sklearn/batch_test_out'

transformer = model.transformer(instance_count=1, 
                                instance_type='ml.m5.xlarge', 
                                output_path=output_path, 
                                assemble_with='Line', 
                                accept='text/csv')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
Using already existing model: byoc-sklearn-2020-11-04-17-32-19-621


In [69]:
WORK_DIRECTORY = '.././DATA'

batch_input = session.upload_data(f'{WORK_DIRECTORY}/batch_test', key_prefix='byoc-sklearn/batch_test')

In [70]:
transformer.transform(batch_input, content_type='text/csv', split_type='Line', input_filter='$')
transformer.wait()

.Gracefully stopping... (press Ctrl+C again to force)
.........................
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [04/Nov/2020 18:01:34] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [04/Nov/2020 18:01:34] "#033GET /execution-parameters HTTP/1.1#033" 404 -
169.254.255.130 - - [04/Nov/2020 18:01:34] "#033POST /invocations HTTP/1.1#033" 200 -
INFO:werkzeug:169.254.255.130 - - [04/Nov/2020 18:01:34] "#033POST /invocations HTTP/1.1#033" 200 -
169.254.255.130 - - [04/Nov/2020 18:01:34] "#033POST /invocations HTTP/1.1#033" 200 -
INFO:werkzeug:169.254.255.130 - - [04/Nov/2020 18:01:34] "#033POST /invocations HTTP/1.1#033" 200 -
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit

#### Inspect Batch Transformed Output

In [80]:
s3_client = session.boto_session.client('s3')
s3_client.download_file(bucket_name, 
                        'byoc-sklearn/batch_test_out/batch_test.csv.out', 
                        '.././DATA/batch_test/batch_test.csv.out')

In [83]:
with open('.././DATA/batch_test/batch_test.csv.out', 'r') as f:
    results = f.readlines()   
    
print("Transform results: \n{}".format(''.join(results)))

Transform results: 
1
3
0
1
1
3
1
3
0
0
0
3
0
0
2

